In [1]:
import torch
import torchvision.io as io

In [2]:
def video_to_tensor(video_path: str, greyscale: bool = False):
    # Read video frames
    video, audio, info = io.read_video(video_path, pts_unit='sec')

    if greyscale:
        # Convert video to tensor
        # Ensure video is grayscale (1 channel)
        if video.shape[1] != 1:
            raise ValueError("Video is not grayscale.")

        video_tensor = video.squeeze(1)  # Remove the channel dimension
        video_tensor = video_tensor.permute(0, 2, 1)  # Swap dimensions to have (time, height, width)
    else:
        # Convert video to tensor
        video_tensor = video.permute(0, 3, 1, 2)  # Swap dimensions to have (time, channels, height, width)

    return video_tensor

In [ ]:
# Example usage
video_path = './videos/Ball_Bounce_Vid.mp4'
video_tensor = video_to_tensor(video_path)
print(video_tensor.shape)